# Amazon Bedrock Multimodal Workshop
## Image clustering with Multimodal Embeddings

In this Jupyter Notebook, we will explore the power of multimodal embeddings to cluster images into different groups. Our goal is to classify images into three categories: kitchen, bedroom, and bathroom. We will leverage the capabilities of vector databases and Amazon Titan Multimodal Embeddings in Amazon Bedrock to achieve this task. Additionally we will make use of Amazon Bedrock Batch Inference to get the embeddings for a larger number of images. 


### Install and import needed libraries

In [ ]:
!pip install -q pinecone-client

In [ ]:
import os
import time
import json
import boto3
import base64
import datetime
from PIL import Image
from utils import resize_image, process_zip
from pinecone import Pinecone, PodSpec

# Boto3 clients
s3_client = boto3.client('s3')
iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
bedrock_client = boto3.client('bedrock')
bedrock_runtime = boto3.client(service_name="bedrock-runtime")

# Account and region info
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]

#### Preview SDK for Batch Inference
At the time of creating this notebook, Batch Inference for Amazon Bedrock is still in public preview. To complete this notebook we will download and install the boto3 and botocore clients versions which include the preview.  

In [ ]:
process_zip("https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip")

In [ ]:
!pip uninstall botocore boto3 -qy  
!pip install -q preview-sdk/botocore-1.32.4-py3-none-any.whl
!pip install -q preview-sdk/boto3-1.29.4-py3-none-any.whl

### Amazon Bedrock Titan Multimodal Embeddings (TMME)
In this section of the notebook we will create the functions needed to retrieve embeddings using Amazon Bedrock TMME. 

#### Define output embedding length
Titan Multimodal Embeddings gives you the option to create embeddings with three vector sizes: 1024, 384 or 256

In [ ]:
outputEmbeddingLength = 1024 # Define output vector size – 1,024 (default), 384, 256

#### Image embeddings
This function will transform an image into an embeddings vector using TMME. 

In [ ]:
def get_embeddings_of_image(image, outputEmbeddingLength = outputEmbeddingLength):
    with open(image, "rb") as image_file:
        imageEncoded = base64.b64encode(image_file.read()).decode('utf8')

    body = json.dumps(
        {
            "inputImage": imageEncoded,
            "embeddingConfig": { 
                "outputEmbeddingLength": outputEmbeddingLength
            }
        }
    )

    response = bedrock_runtime.invoke_model(
        body=body,
        modelId="amazon.titan-embed-image-v1",
        accept="application/json",
        contentType="application/json"
    )

    vector = json.loads(response['body'].read().decode('utf8'))
    return vector

In [ ]:
### Download and prepare the dataset

### Pinecone Vector Database

### Store embeddings in a vector database
Now we know how to get embeddings for our content, we are going to store them in a vector database to later on query and retrieve results. 

In this case we are going to use Pinecone.

To complete this section you will need to create a free account with Pinecone which includes a free index to test and retrieve your API key. 

#### Create the vector database

In [ ]:
pinecone =  Pinecone(api_key="Replace with Your Pinecone API key")

In [ ]:
index_name = "house-rooms"

In [ ]:
pinecone.create_index(index_name,
        dimension=outputEmbeddingLength,
        metric='cosine',
        spec=PodSpec(environment="gcp-starter"))

In [ ]:
pinecone.describe_index(index_name)

### Store images into the vector database -- Index 3 images (clustering groups)
Now our Pinecone index is created, we can start ingesting our images embeddings. 

In [ ]:
index = pinecone.Index(index_name)

In [ ]:
def send_info_to_vectordb(vector, image, image_name, type, index):
    index.upsert([
        (image_name, vector["embedding"], {"path": image, "type": type})
    ])

In [ ]:
images_folder = "base-images"
for image_name in os.listdir(images_folder):
    if image_name.endswith(".jpg"):
        image_path = os.path.join(images_folder, image_name)
        imagename_without_extension = os.path.splitext(image_name)[0]
        type = imagename_without_extension.split("_")[0]
        image = Image.open(image_path)
        if (image.size[0] > 2048 or image.size[1] > 2048):
            resize_image(image_path)
        print("Indexing:", image_path)
        vector = get_embeddings_of_image(image_path)
        send_info_to_vectordb(vector, image_path, imagename_without_extension, type, index)

### Compare a sample image to retrieve classification

#### Image query
With this function we will first transform our image query into an embeddings vector, which we will then use to query the vector database. 

In [ ]:
def query_the_database_with_image(image):
    vector = get_embeddings_of_image(image)["embedding"]
    results = index.query(
        vector=vector,
        top_k=1,
        include_metadata=True,
        include_values=True
    )
    return results

#### Cluster the test images
With this function we will first cluster some test images.

In [ ]:
def test_images_classification():
    images_folder = "test-images"
    for image_name in os.listdir(images_folder):
       if image_name.endswith(".jpg"):
           image_path = os.path.join(images_folder, image_name)
           image = Image.open(image_path)
           if (image.height > 2048 or image.width > 2048):
                resize_image(image_path)
           results = query_the_database_with_image(image_path)
           #print(results)
           print("Photo: {} is type {}".format(image_path, results["matches"][0]["metadata"]["type"]))

In [ ]:
test_images_classification()

### Create embeddings at large scale with Amazon Bedrock Batch Inference

#### Create an Amazon S3 bucket
Create an bucket where your input/output data will be stored.

If you already have a bucket created, replace the name in the next cell and skip the following cell. 

In [ ]:
bucket_name = "amazonbr-batch-embeddings-{}-{}".format(account_id, region)
s3_bucket_path = "s3://{}".format(bucket_name)

In [ ]:
try:
    if region != 'us-east-1':
        s3_client.create_bucket(
            Bucket=bucket_name,     
            CreateBucketConfiguration={
                'LocationConstraint': region
            },
        )
    else:
        s3_client.create_bucket(Bucket=bucket_name)
    print("AWS Bucket: {}".format(bucket_name))
except Exception as err:
    print("ERROR: {}".format(err))

s3_bucket_path = "s3://{}".format(bucket_name)
print("S3 bucket path: {}".format(s3_bucket_path))

### Batch inference preparation - Creating role and policies requirements

We will now prepare the necessary role for the batch inference job. That includes creating the policies required to run model invocation jobs with Amazon Bedrock.

#### Create Trust relationship
This JSON object defines the trust relationship that allows the bedrock service to assume a role that will give it the ability to talk to other required AWS services. The conditions set restrict the assumption of the role to a specfic account ID and a specific component of the bedrock service (model_invocation_jobs)

In [ ]:
role_name = "AmazonBedrockModelInvocation-batch-embeddings"
s3_bedrock_ft_access_policy="AmazonBedrock-batch-embeddings"
embeddings_model_arn= "arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-embed-image-v1"
embeddings_model_arn_2 = "arn:aws:bedrock:us-east-1:947565228676:custom-model/amazon.titan-embed-image-v1:0/wrmjvnf7a922"

In [ ]:
ROLE_DOC = f"""{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Effect": "Allow",
            "Principal": {{
                "Service": "bedrock.amazonaws.com"
            }},
            "Action": "sts:AssumeRole",
            "Condition": {{
                "StringEquals": {{
                    "aws:SourceAccount": "{account_id}"
                }},
                "ArnEquals": {{
                    "aws:SourceArn": "arn:aws:bedrock:{region}:{account_id}:model-invocation-job/*"
                }}
            }}
        }}
    ]
}}
"""

### Create S3 access policy

This JSON object defines the permissions of the role we want bedrock to assume to allow access to the S3 bucket that we created that will hold our prompts and allow certain bucket and object manipulations.


In [ ]:
ACCESS_POLICY_DOC = f"""{{
    "Version": "2012-10-17",
    "Statement": [
        {{
            "Effect": "Allow",
            "Action": [
                "s3:AbortMultipartUpload",
                "s3:DeleteObject",
                "s3:PutObject",
                "s3:GetObject",
                "s3:GetBucketAcl",
                "s3:GetBucketNotification",
                "s3:ListBucket",
                "s3:PutBucketNotification"
            ],
            "Resource": [
                "arn:aws:s3:::{bucket_name}",
                "arn:aws:s3:::{bucket_name}/*"
            ]
        }}
    ]
}}"""

### Create IAM role and attach policies

Let's now create the IAM role with the created trust policy and attach the s3 policy to it

In [ ]:
response = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=ROLE_DOC,
    Description="Role for Bedrock to access S3 for model invocation",
)

In [ ]:
role_arn = response["Role"]["Arn"]
response = iam_client.create_policy(
    PolicyName=s3_bedrock_ft_access_policy,
    PolicyDocument=ACCESS_POLICY_DOC,
)
policy_arn = response["Policy"]["Arn"]
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn,
)

### Configure the model invocation job
#### Create the input dataset

In [ ]:
folder_path = "batch-images"
input_key = "input.jsonl"
output_path = "validation/output/"

def image_to_base64(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

data = []

# Supported image file extensions
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif']

# Iterate over files in the folder
for i, file_name in enumerate(os.listdir(folder_path)):
    file_path = os.path.join(folder_path, file_name)

    _, extension = os.path.splitext(file_path)
    if extension.lower() in image_extensions:
        image = Image.open(file_path)
        if image.height > 2048 or image.width > 2048:
            resize_image(file_path)

        model_input = {
            "inputImage": image_to_base64(file_path),
            "embeddingConfig": {
                "outputEmbeddingLength": outputEmbeddingLength 
            }
        }

        data.append({'recordId': file_name, 'modelInput': model_input})

In [ ]:
print("Input data items are:", len(data))

#### Process data and output to new lines
The model invocation job requires the input data to be in jsonl format and located Amazon S3.

In [ ]:
output_data = ""
for row in data:
    output_data += json.dumps(row) + "\n"

s3_client.put_object(Body=output_data, Bucket=bucket_name, Key=input_key)

#### Define data configuration and launch job
As the input data is prepared and uploaded to Amazon S3 we can go ahead and launch the invocation job.

In [ ]:
inputDataConfig=({
    "s3InputDataConfig": {
        "s3InputFormat": "JSONL",
        "s3Uri": "{}/{}".format(s3_bucket_path, input_key)
    }
})

outputDataConfig=({
    "s3OutputDataConfig": {
        "s3Uri": "{}/{}".format(s3_bucket_path, output_path)
    }
})
date_time = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

response = bedrock_client.create_model_invocation_job(
    roleArn=role_arn,
    modelId=embeddings_model_arn_2,
    jobName=f"my-batch-job-test-{date_time}",
    inputDataConfig=inputDataConfig,
    outputDataConfig=outputDataConfig
)

jobArn = response.get('jobArn')

In [ ]:
%%time
status = bedrock_client.get_model_invocation_job(jobIdentifier=jobArn)['status']
while status not in ["Completed", "Failed", "Stopping", "Stopped"]:
    status = bedrock_client.get_model_invocation_job(jobIdentifier=jobArn)['status']
    print(status)
    time.sleep(30)

#### Retrieve the embeddings 
Once the batch job is complete we can go ahead and download the output file and extract the embeddings. 

In [ ]:
job_id = jobArn.split("/")[-1]
images_file = "input.jsonl.out"
s3_client.download_file(bucket_name, "{}{}/{}".format(output_path, job_id, images_file), images_file)

In [ ]:
record_embedding_list = []
with open('input.jsonl.out', 'r') as file:
    for line in file:
        data = json.loads(line)
        record_id = data['recordId']
        embedding = data['modelOutput']['embedding']
        record_embedding = {'recordId': record_id, 'embedding': embedding}
        record_embedding_list.append(record_embedding)
print("The embedding list contains {} records.".format(len(record_embedding_list)))

### Clean Up
In this section we will delete any resource which may incur in unnecessary costs.

#### Delete the Amazon S3 bucket

In [ ]:
# Delete all objects in the bucket
try:
    response = s3_client.list_objects_v2(Bucket=bucket_name)
    if 'Contents' in response:
        for obj in response['Contents']:
            s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
        print(f"All objects in {bucket_name} have been deleted.")
except Exception as e:
    print(f"Error deleting objects from {bucket_name}: {e}")

# Delete the bucket
try:
    response = s3_client.delete_bucket(Bucket=bucket_name)
    print(f"Bucket {bucket_name} has been deleted.")
except Exception as e:
    print(f"Error deleting bucket {bucket_name}: {e}")

#### Delete the Pinecone Index

In [ ]:
pc.delete_index(index_name)